### RFC

### import modules and configure notebook

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)
%matplotlib inline

### Load variables stored by data_preproccessing notebook

In [2]:
%store -r train_data_formodel
%store -r test_data
%store -r my_data
%store -r uniques
%store -r best_feats

### configurations
* save_plots -> boolean
* random_seed_state -> number, sets random state for model and for stratified splits 
* pickle_model -> True|False, wether model should be serialised and saved
* pickle_model_name -> string, name of serialised model
* scale -> True|False if set to True then features scaled to all have mean value 0 and standard deviation 1
* pickle_file_path -> string,  filepath for serialised model to be saved to

In [3]:
save_plots = False
random_seed_state = 42
pickle_model = False
pickle_model_name = 'grouped'
pickle_file_path = '../../../model'
scale = False

### counts of instances in all classes before oversampling

In [4]:
train_data_formodel['class'].value_counts()

4     105
18    100
19     61
0      53
10     47
13     45
16     36
17     36
2      36
12     30
11     30
8      30
7      30
5      30
6      27
9      27
1      24
14     21
3      18
15     15
Name: class, dtype: int64

### The class column is stored as the variable y 

In [5]:
y_train = np.array(train_data_formodel['class'])

### The variables identified as best by feature selection are used as features

In [6]:
train_data_feats = train_data_formodel[best_feats]
test_data_feats = test_data[best_feats]
test_data_identifiers = test_data['Analysis']

### features are standardised

In [7]:
if scale:
    my_scaler = StandardScaler()
    X_test = np.array(my_scaler.fit_transform(test_data_feats))
    X_train = np.array(my_scaler.fit_transform(train_data_feats))
else:
    X_test = np.array(test_data_feats)
    X_train = np.array(train_data_feats)

### the dimensions of the class and features are checked

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(801, 30)
(363, 30)
(801,)


### Local Outlier Factor model is fitted on data that will be used for training the final classifier model

In [9]:
lof = LocalOutlierFactor(novelty=True)

### print details of model

In [10]:
print(lof)

LocalOutlierFactor(algorithm='auto', contamination='legacy', leaf_size=30,
          metric='minkowski', metric_params=None, n_jobs=None,
          n_neighbors=20, novelty=True, p=2)


In [11]:
lof.fit(X_train)

/Users/robbie.morse/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


LocalOutlierFactor(algorithm='auto', contamination='legacy', leaf_size=30,
          metric='minkowski', metric_params=None, n_jobs=None,
          n_neighbors=20, novelty=True, p=2)

### Local Outlier Factor model predicts whether the artefact samples are inliers or outliers encoded with 1 and -1 respectively

In [12]:
predictions = lof.predict(X_test)

In [13]:
pd.Series(predictions).value_counts()

 1    258
-1    105
dtype: int64

In [14]:
nInliers = list(pd.Series(predictions).value_counts())[0]
nOutliers = list(pd.Series(predictions).value_counts())[1]

In [15]:
print('there are {0} inliers and {1} outliers, so the proportion of inliers is {2}'.format(nInliers, nOutliers, (nInliers)/(X_test.shape[0])))

there are 258 inliers and 105 outliers, so the proportion of inliers is 0.7107438016528925


### column encoding inlier status is added to artefact-sample dataset 

In [16]:
X_test_df = pd.DataFrame(data = X_test, columns = test_data_feats.columns.values)

In [17]:
X_test_labeled_df = pd.concat([test_data_identifiers.reset_index(drop = True), X_test_df, pd.Series(predictions)], axis =1).rename(columns = {0:'inlierLabel'})

In [18]:
X_test_labeled_df.head()

,Analysis,Li7,Nd146,Pr141,Ba137,Sr88,Rb85,Ge72,Cu63,Fe56,Cr52,V51,Zr90,Sc45,B11,Mg24,Al27,P31,S33,U238,La139,Ga69,Y89,K39,Ce140,Cd111,Zn68,Ni60,Nb93,Si28,Mn55,inlierLabel
0,06_DH1_1,2.79,0.34,0.07,5.95,2.00,0.720000,1.37,11.14,35.38,5.18,0.51,1.43,1.08,93.21,27.83,635.240000,35.07,806.55,0.309799,0.18,0.69,0.21,443.100000,0.57,0.207143,20.75,1.03,0.222487,465388.29,3.27,1
1,07_DH1_2,2.37,0.24,0.06,7.51,1.84,0.770000,1.93,8.64,49.23,5.43,0.60,1.47,1.09,95.19,27.99,639.090000,31.65,807.55,0.309799,0.19,0.86,0.21,442.100000,0.53,0.207143,18.15,1.22,0.222487,465388.40,2.30,1
2,08_DH1_3,2.46,0.08,0.03,4.40,1.93,0.710000,2.14,3.44,16.46,2.61,0.62,1.16,1.17,93.63,26.43,670.480000,25.43,837.73,0.309799,0.11,0.79,0.13,444.280000,0.20,0.207143,17.54,1.01,0.222487,465378.49,1.40,1
3,09_DH2_1,18.47,0.25,0.06,12.48,4.29,1.620000,1.59,1.10,16.76,2.87,1.52,1.45,1.19,63.80,31.98,620.082431,24.63,750.62,0.080000,0.23,0.75,0.20,383.119348,0.40,0.220000,14.33,1.07,0.250000,464555.55,1.09,1
4,10_DH2_2,19.98,0.51,0.12,14.40,4.44,0.735373,0.81,1.23,62.16,5.92,2.44,1.99,1.22,60.96,44.95,620.082431,22.59,796.21,0.090000,0.48,0.76,0.27,383.119348,0.80,0.090000,14.59,0.93,0.680000,464137.82,1.26,1


### dataset is stored

In [19]:
%store X_test_labeled_df

Stored 'X_test_labeled_df' (DataFrame)
